In [1]:
import os
import pandas as pd
import yaml
import configparser

# Functions

In [188]:
# generate a list of legislators that have twitter handles
def parse_twitter_handles(config):
    yaml_doc_path = config.get('data', 'path') + '/' + config.get('data', 'twitter_yaml')
    if not os.path.isfile(yaml_doc_path):
        fetch_twitter_handles()
    yaml_doc = yaml.load(open(yaml_doc_path, 'r'))
    return [d for d in yaml_doc if 'twitter' in d['social']]

# create a dict where the key is the date and the value is a 
# list of lists [[moc, score],[moc, score],...] for that day
def parse_moc_scores(config, score_files):
    moc_scores = {}
    for f in score_files:
        score_path = config.get('data', 'path') + '/' + config.get('data', 'scores') + '/' + f
        if '.1.' in f:
            components = f.split('.')
            date = components[0]
            if components[2] == 'moc':
                with open(score_path) as score_file:
                    scores = score_file.readlines()
                moc_scores[date] = scores
    return moc_scores

# create a dict of scores for each legislator
# on each date for which scores are available
def read_scores(legislators, scores):
    leg_scores = {}
    for l in legislators:
        handle = l['social']['twitter'].lower()
        leg_scores[handle] = {}

    for date in scores:
        for score_item in scores[date]:
            handle, score = score_item.split()
            leg_scores[handle][date] = score
    return leg_scores

# main()

In [189]:
config = configparser.ConfigParser()
config.read('settings.cfg')
#score_path = config.get('data', 'path') + '/' + config.get('data', 'scores') + '/'
#score_files = os.listdir(score_path)
score_files = 'data/scores'

moc_scores = parse_moc_scores(config, score_files)
mocs_with_twitter = parse_twitter_handles(config)
moc_scores_by_date = read_scores(mocs_with_twitter, moc_scores)
#df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in moc_scores_by_date.items()])).transpose().dropna(how='all')
#data_file = config.get('data', 'path') + '/' + config.get('data', 'moc_scores')
#df.to_csv(data_file, sep='\t')

In [190]:
mocs_with_twitter

[{'id': {'bioguide': 'R000600', 'govtrack': 412664, 'thomas': '02222'},
  'social': {'facebook': 'congresswomanaumuaamata',
   'facebook_id': '1537155909907320',
   'twitter': 'RepAmata',
   'twitter_id': 3026622545,
   'youtube_id': 'UCGdrLQbt1PYDTPsampx4t1A'}},
 {'id': {'bioguide': 'H001070', 'govtrack': 412645, 'thomas': '02260'},
  'social': {'facebook': 'RepCresentHardy',
   'facebook_id': '320612381469421',
   'twitter': 'RepHardy',
   'twitter_id': 2964222544,
   'youtube': 'RepHardy',
   'youtube_id': 'UCc8E6NWCdgrXjBVI2NNPYdA'}},
 {'id': {'bioguide': 'Y000064', 'govtrack': 412428, 'thomas': '02019'},
  'social': {'facebook': 'RepToddYoung',
   'facebook_id': '186203844738421',
   'twitter': 'RepToddYoung',
   'twitter_id': 234128524,
   'youtube': 'RepToddYoung',
   'youtube_id': 'UCuknj4PGn91gHDNAfboZEgQ'}},
 {'id': {'bioguide': 'E000295', 'govtrack': 412667, 'thomas': '02283'},
  'social': {'facebook': 'senjoniernst',
   'facebook_id': '351671691660938',
   'instagram': 'sen

Now read data file saved from AWS server and do next steps for analysis.

In [1]:
full_df = pd.read_csv('moc_scores_by_date.csv', sep = '\t')

NameError: name 'pd' is not defined

Labor Day 2016 - Monday September 5. Tuesday, September 6 is 9 weeks before election. 2016-09-

In [ ]:
all_dates = list(full_df.columns.values)
len(all_dates)

In [ ]:
last_date = all_dates[365]
last_date

In [ ]:
import datetime
date_parts = last_date.strip('-')

start_date = datetime.date(2016, 9, 6)
end_date = datetime.date(2016, 11, 8)

In [ ]:
date_parts

In [ ]:
len(last_date)

In [ ]:
year = int(last_date[:4])
month = int(last_date[5:7])
day = int(last_date[8:])

In [ ]:
interesting_dates = []
for date in all_dates:
    try:
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:])
    except:
        continue
    
    date_in_question = datetime.date(year, month, day)
    if start_date <= date_in_question <= end_date:
        interesting_dates.append(date)

In [ ]:
interesting_df = full_df.filter(interesting_dates)
interesting_df.transpose()
#interesting_df.to_csv('purpletag_moc_scores_election_2016_transposed.csv', sep='\t')

# Legislator Info

In [170]:
import yaml
yaml_doc = yaml.load(open('legislators-current.yaml', 'r'))
#return [d for d in yaml_doc if 'twitter' in d['social']]

In [171]:
#yaml_doc[0]

In [172]:
name = yaml_doc[0]['name']['official_full']
chamber = yaml_doc[0]['terms'][-1]['type']
state = yaml_doc[0]['terms'][-1]['state']

if yaml_doc[-1]['terms'][-1]['type'] == 'rep':
    district = yaml_doc[-1]['terms'][-1]['district']
else:
    district = 99

for x in reversed(range(len(yaml_doc[0]['terms']))):
    if yaml_doc[0]['terms'][x]['type'] == chamber:
        start = yaml_doc[0]['terms'][x]['start']
        
party = yaml_doc[0]['terms'][-1]['party']
url = yaml_doc[0]['terms'][-1]['url']

legislator = {'name': name, 'chamber': chamber, 'state': state, 'district': district, 'start_date': start, 'party': party, 'url': url}

In [173]:
legislator

{'chamber': 'sen',
 'district': 2,
 'name': 'Sherrod Brown',
 'party': 'Democrat',
 'start_date': '2007-01-04',
 'state': 'OH',
 'url': 'https://www.brown.senate.gov'}

In [199]:
all_reps = []
for item in yaml_doc:
    govtrack_id = item['id']['govtrack']
    name = item['name']['official_full']
    chamber = item['terms'][-1]['type']
    state = item['terms'][-1]['state']
    # only house has district, senators are assigned district 99
    if item['terms'][-1]['type'] == 'rep':
        district = item['terms'][-1]['district']
    else:
        district = 99
    # start date of first term in current chamber
    for x in reversed(range(len(item['terms']))):
        if item['terms'][x]['type'] == chamber:
            start = item['terms'][x]['start']
    party = item['terms'][-1]['party']
    # not all reps have a url
    if 'url' in item['terms'][-1]:
        url = item['terms'][-1]['url']
    else:
        url = 'none'
    legislator = {'id': govtrack_id, 'name': name, 'chamber': chamber, 'state': state, 'district': district, 'start_date': start, 'party': party, 'url': url}
    all_reps.append(legislator)

In [200]:
#mocs_with_twitter

In [208]:
all_reps
#mocs_with_twitter
for moc in mocs_with_twitter:
    #govtrack_id = moc['id']
    try:
        govtrack_id = moc['id']['govtrack']
    except KeyError as e:
        govtrack_id = 'NA'
    for rep in all_reps:
        if govtrack_id == rep['id']:
            twitter = moc['social']['twitter']
            #print(twitter)
            rep['twitter'] = twitter.lower()
            #print(rep['twitter'])

chamber = 'sen' or 'rep'
start_date = 'yyyy-mm-dd'
party = 'Democrat' or 'Republican'

In [211]:
all_reps

[{'chamber': 'sen',
  'district': 99,
  'id': 400050,
  'name': 'Sherrod Brown',
  'party': 'Democrat',
  'start_date': '2007-01-04',
  'state': 'OH',
  'twitter': 'sensherrodbrown',
  'url': 'https://www.brown.senate.gov'},
 {'chamber': 'sen',
  'district': 99,
  'id': 300018,
  'name': 'Maria Cantwell',
  'party': 'Democrat',
  'start_date': '2001-01-03',
  'state': 'WA',
  'twitter': 'senatorcantwell',
  'url': 'https://www.cantwell.senate.gov'},
 {'chamber': 'sen',
  'district': 99,
  'id': 400064,
  'name': 'Benjamin L. Cardin',
  'party': 'Democrat',
  'start_date': '2007-01-04',
  'state': 'MD',
  'twitter': 'senatorcardin',
  'url': 'https://www.cardin.senate.gov'},
 {'chamber': 'sen',
  'district': 99,
  'id': 300019,
  'name': 'Thomas R. Carper',
  'party': 'Democrat',
  'start_date': '2001-01-03',
  'state': 'DE',
  'twitter': 'senatorcarper',
  'url': 'http://www.carper.senate.gov'},
 {'chamber': 'sen',
  'district': 99,
  'id': 412246,
  'name': 'Robert P. Casey, Jr.',
  '

need to map this list of legislators to a list of twitter handles, not every rep uses twitter

In [ ]:
#TODO

# Time Series Analysis

In [ ]:
%matplotlib inline
import os  
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
#import statsmodels.api as sm  
#import seaborn as sb  
#sb.set_style('darkgrid')

path = os.getcwd() + '/purpletag_moc_scores_election_2016_transposed.csv'  
election_data = pd.read_csv(path, sep = '\t')  
#stock_data['Date'] = stock_data[0].convert_objects(convert_dates='coerce')  
#stock_data = stock_data.sort_index(by='Date')  
#stock_data = stock_data.set_index('Date')  

In [ ]:
election_data

del election_data['Unnamed: 0'] # remove extra column from cached file

In [ ]:
#election_data

In [ ]:
new_election_data = pd.DataFrame(election_data.transpose())
#dates = list(new_election_data.index)
#new_election_data['Date'] = dates
#new_election_data

In [ ]:
dates = pd.date_range('2016-09-06', '2016-11-08', freq='D')

In [ ]:
#dates

In [ ]:
#new_election_data - removed from next line: convert_dates='coerce'
#new_election_data['Date'] #= new_election_data['Date'].convert_objects()  
#new_election_data = new_election_data.sort_index(by='Date')  
#new_election_data = new_election_data.set_index('Date')  

In [ ]:
#new_election_data

In [ ]:
#new_election_data[2].plot(figsize=(16, 12))  
#new_election_data.plot(figsize=(16, 12))  

In [ ]:
#data = pd.read_csv('AirPassengers.csv')
print(new_election_data.head())
print('\n Data Types:')
print(new_election_data.dtypes)

In [ ]:
dates = new_election_data.index

In [ ]:
dates

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
#data = new_election_data()
data = pd.read_csv('purpletag_moc_scores_election_2016_transposed.csv', parse_dates='Month', index_col='Month',date_parser=dateparse)
print(data.head())

There is a peak of - scores on September 11. 

In [ ]:
new_election_data['First Difference'] = new_election_data[2] - new_election_data[2].shift()  
new_election_data['First Difference'].plot(figsize=(16, 12)) 

In [ ]:
new_election_data['Natural Log'] = new_election_data[2].apply(lambda x: np.log(x))  
new_election_data['Natural Log'].plot(figsize=(16, 12)) 

# Another Try

Here's a dataframe where each row is a "time series" of one congressional rep scores over time.

In [ ]:
#interesting_df

In [ ]:
from datetime import datetime
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as pyplot

In [ ]:
dates = list(interesting_df)

In [ ]:
#dates

In [ ]:
row1 = interesting_df.iloc[0]

In [ ]:
#scores = row1.tolist()
#scores

In [ ]:
#data = {'date': dates, 'score': scores}
#df = pd.DataFrame(data, columns = ['date', 'score'])
#print(df)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
#df.index = df['date']
#del df['date']
#df

In [ ]:
#df['2016-11']

In [ ]:
#df.groupby(level=0).count()

In [ ]:
#df.plot()

plot scores over time over dates before election.

In [ ]:
%matplotlib inline
twisted = interesting_df.transpose()
twisted.plot(legend=False, figsize=(27, 18))

Now plot average scores over 7-day windows

In [ ]:
#twisted.plot(legend=False, kind="scatter", figsize=(900, 600))

In [ ]:
import plotly

plotly.offline.plot(twisted)

In [ ]:
interesting_df.rows

# Purpletag Parse

In [142]:
score_path = "data/scores/"
filenames = os.listdir(score_path)
#filenames

In [143]:
moc_scores = {}
for f in filenames:
    if '.1.' in f:
        components = f.split('.')
        date = components[0]
        score_filename = score_path + f
        if components[2] == 'moc':
            with open(score_filename) as score_file:
                scores = score_file.readlines()
                moc_scores[date] = scores

In [144]:
#moc_scores

In [145]:
from datetime import datetime as dt
from datetime import timedelta as td
relevant_scores = {}
start_date = dt(2016, 9, 6)
end_date = dt(2016, 11, 8)
print(start_date)

for date in moc_scores:
    year, month, day = date.split('-')
    score_day = dt(int(year), int(month), int(day))
    if start_date <= score_day <= end_date:
        relevant_scores[date] = moc_scores[date]


2016-09-06 00:00:00


In [146]:
week1 = start_date + td(days=6)
week1

datetime.datetime(2016, 9, 12, 0, 0)

In [147]:
week2 = week1 + td(days=7)
week3 = week2 + td(days=7)
week4 = week3 + td(days=7)
week5 = week4 + td(days=7)
week6 = week5 + td(days=7)
week7 = week6 + td(days=7)
week8 = week7 + td(days=7)
week9 = week8 + td(days=7)

In [148]:
week9

datetime.datetime(2016, 11, 7, 0, 0)

In [149]:
# all_scores is dictionary of date keys and list of moc scores as values
# start date is beginning of week
# end date is end of week
# returns dictionary of moc keys and avg weekly score 
def get_average_weekly_scores(all_scores, start_date, end_date):
    weekly_scores = {}
    for date in all_scores:
        year, month, day = date.split('-')
        score_day = dt(int(year), int(month), int(day))
        if start_date <= score_day <= end_date:
            weekly_scores[date] = all_scores[date]
            
    moc_scores = {}
    for date in weekly_scores:
        for moc in weekly_scores[date]:
            line = moc.strip()
            handle, score = line.split(' ')
            if handle in moc_scores:
                moc_scores[handle].append(score)
            else:
                moc_scores[handle] = [score]
                
    avg_moc_scores = {}
    for moc in moc_scores:
        score_list = moc_scores[moc]
        length = len(score_list)
        score_sum = 0
        for score in score_list:
            score_sum += float(score)
        avg = score_sum / length
        avg_moc_scores[moc] = avg
        
    return avg_moc_scores

In [150]:
#moc_scores

In [151]:
week1_scores = get_average_weekly_scores(relevant_scores, start_date, week1)
week2_scores = get_average_weekly_scores(relevant_scores, week1+td(days=1), week2)
week3_scores = get_average_weekly_scores(relevant_scores, week2+td(days=1), week3)
week4_scores = get_average_weekly_scores(relevant_scores, week3+td(days=1), week4)
week5_scores = get_average_weekly_scores(relevant_scores, week4+td(days=1), week5)
week6_scores = get_average_weekly_scores(relevant_scores, week5+td(days=1), week6)
week7_scores = get_average_weekly_scores(relevant_scores, week6+td(days=1), week7)
week8_scores = get_average_weekly_scores(relevant_scores, week7+td(days=1), week8)
week9_scores = get_average_weekly_scores(relevant_scores, week8+td(days=1), week9)



In [152]:
election_scores = {}
for moc in week1_scores:
    election_scores[moc] = [week1_scores[moc]]

In [153]:
for moc in week2_scores:
    if moc in election_scores:
        election_scores[moc].append(week2_scores[moc])
    else:
        election_scores[moc] = [week2_scores[moc]]

In [158]:
def add_weekly_scores(all_scores, weekly_scores):
    for moc in all_scores:
        print(moc)
        for key in moc:
            if key in weekly_scores:
                moc[key].append(weekly_scores[key])
            else:
                print(moc)
                moc[key].append(0)
    return all_scores

In [161]:
#election_scores

In [159]:
election_scores = add_weekly_scores(election_scores, week3_scores)

reprichmond
reprichmond


TypeError: string indices must be integers

In [73]:
election_scores = add_weekly_scores(election_scores, week4_scores)
election_scores = add_weekly_scores(election_scores, week5_scores)
election_scores = add_weekly_scores(election_scores, week6_scores)
election_scores = add_weekly_scores(election_scores, week7_scores)
election_scores = add_weekly_scores(election_scores, week8_scores)
election_scores = add_weekly_scores(election_scores, week9_scores)

In [74]:
election_scores

{'austinscottga08': [4.932035, 1.06429, 3.6757, 6.0426, 1.56161],
 'benniegthompson': [-1.21138, -1.82301],
 'bettymccollum04': [-56.669016,
  -119.88476,
  -67.1729,
  -59.540265000000005,
  -27.99579833333333,
  -33.955594999999995,
  -4.8796734,
  -33.08063249999999,
  -56.5287],
 'billpascrell': [-2.87143,
  -2.6238099999999998,
  -2.353348,
  -2.3378966666666665,
  -1.02655,
  -1.33333,
  -17.9542],
 'boblatta': [1.37997, 9.673526666666666, 0.974138],
 'bradsherman': [-1.94118, -5.38195, -0.748092],
 'call_me_dutch': [-1.31258278,
  -4.765343333333333,
  -3.129013333333333,
  -1.57327019,
  -2.967365,
  -2.7154760000000002,
  -20.466664333333334,
  -1.5766299999999998,
  -6.73282],
 'candicemiller': [1.678195, 0.0296569, -0.969076, 1.08696],
 'cathymcmorris': [7.926214999999999,
  3.1412825,
  8.87357,
  13.830632,
  1.0,
  7.823812500000001,
  3.7478775,
  11.442992000000002,
  10.554400428571428],
 'cbrangel': [-39.434356666666666,
  -61.711175,
  -41.164823999999996,
  -37.3339

In [162]:
all_reps = []
for rep in mocs_with_twitter:
    rep_handle = rep['social']['twitter']
    rep_handle = rep_handle.lower()
    all_reps.append({rep_handle:[]})

In [163]:
len(all_reps)

525

In [164]:
for rep in all_reps:
    for key, value in rep.items():
        key = key.lower()

In [165]:
week1_reps = add_weekly_scores(all_reps, week1_scores)

{'repamata': []}
{'repamata': []}
{'rephardy': []}
{'reptoddyoung': []}
{'senjoniernst': []}
{'senthomtillis': []}
{'repkevinyoder': []}
{'repjohnyarmuth': []}
{'repdonyoung': []}
{'rep_stevewomack': []}
{'repwilson': []}
{'repwebster': []}
{'repwebster': []}
{'markwarner': []}
{'robwittman': []}
{'senwhitehouse': []}
{'peterwelch': []}
{'repwalberg': []}
{'repdwstweets': []}
{'repwestmoreland': []}
{'repjoewilson': []}
{'repgregwalden': []}
{'ronwyden': []}
{'senatorwicker': []}
{'repedwhitfield': []}
{'repedwhitfield': []}
{'maxinewaters': []}
{'maxinewaters': []}
{'chrisvanhollen': []}
{'repvisclosky': []}
{'repvisclosky': []}
{'nydiavelazquez': []}
{'senatortomudall': []}
{'repfredupton': []}
{'reptipton': []}
{'reppaultonko': []}
{'repdinatitus': []}
{'congressmangt': []}
{'nikiinthehouse': []}
{'senatortester': []}
{'repmiketurner': []}
{'tiberipress': []}
{'tiberipress': []}
{'sentoomey': []}
{'repthompson': []}
{'senjohnthune': []}
{'mactxpress': []}
{'benniegthompson': []}
{'b

In [166]:
week2_reps = add_weekly_scores(week1_reps, week2_scores)

{'repamata': [0]}
{'repamata': [0]}
{'rephardy': [2.881545]}
{'reptoddyoung': [1.3345266666666669]}
{'senjoniernst': [3.699435]}
{'senthomtillis': [-8.291870833333334]}
{'repkevinyoder': [0.7463941132]}
{'repjohnyarmuth': [-60.222445400000005]}
{'repdonyoung': [-3.50596]}
{'repdonyoung': [-3.50596]}
{'rep_stevewomack': [1.7680525999999996]}
{'repwilson': [-58.5145325]}
{'repwebster': [0]}
{'repwebster': [0]}
{'markwarner': [-17.073692600000005]}
{'robwittman': [4.340949999999999]}
{'robwittman': [4.340949999999999]}
{'senwhitehouse': [-26.853336000000002]}
{'peterwelch': [-2.528465]}
{'repwalberg': [-2.9660825000000006]}
{'repdwstweets': [-31.905462250000003]}
{'repwestmoreland': [1.4520978000000002]}
{'repjoewilson': [3.4654526666666663]}
{'repgregwalden': [-11.1850635]}
{'repgregwalden': [-11.1850635]}
{'ronwyden': [-23.538565000000002]}
{'senatorwicker': [3.311616]}
{'repedwhitfield': [0]}
{'repedwhitfield': [0]}
{'maxinewaters': [0]}
{'maxinewaters': [0]}
{'chrisvanhollen': [-54.96

In [167]:
week3_reps = add_weekly_scores(week2_reps, week3_scores)
week4_reps = add_weekly_scores(week3_reps, week4_scores)
week5_reps = add_weekly_scores(week4_reps, week3_scores)
week6_reps = add_weekly_scores(week5_reps, week6_scores)
week7_reps = add_weekly_scores(week6_reps, week7_scores)
week8_reps = add_weekly_scores(week7_reps, week8_scores)
week9_reps = add_weekly_scores(week8_reps, week9_scores)

{'repamata': [0, 0]}
{'repamata': [0, 0]}
{'rephardy': [2.881545, 10.322063000000002]}
{'reptoddyoung': [1.3345266666666669, 5.624212750000001]}
{'senjoniernst': [3.699435, 4.20655]}
{'senthomtillis': [-8.291870833333334, 7.017763333333334]}
{'repkevinyoder': [0.7463941132, 8.088735]}
{'repjohnyarmuth': [-60.222445400000005, -64.71197333333333]}
{'repdonyoung': [-3.50596, 0]}
{'repdonyoung': [-3.50596, 0]}
{'rep_stevewomack': [1.7680525999999996, 3.088811666666667]}
{'repwilson': [-58.5145325, -77.1195525]}
{'repwebster': [0, 0]}
{'repwebster': [0, 0]}
{'markwarner': [-17.073692600000005, -39.421077499999996]}
{'robwittman': [4.340949999999999, 0]}
{'senwhitehouse': [-26.853336000000002, -27.274468333333335]}
{'peterwelch': [-2.528465, -3.2126]}
{'repwalberg': [-2.9660825000000006, 3.7697406]}
{'repdwstweets': [-31.905462250000003, -45.32210249999999]}
{'repwestmoreland': [1.4520978000000002, 9.113617999999999]}
{'repwestmoreland': [1.4520978000000002, 9.113617999999999]}
{'repjoewilso

In [168]:
week9_reps

[{'repamata': [0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'rephardy': [2.881545,
   10.322063000000002,
   5.229936666666667,
   3.146738,
   5.229936666666667,
   3.3147416666666665,
   2.3427266666666666,
   4.379330250000001,
   2.3421875]},
 {'reptoddyoung': [1.3345266666666669,
   5.624212750000001,
   3.0671,
   2.01617,
   3.0671,
   5.801352666666666,
   0.989362,
   0.912281,
   0]},
 {'senjoniernst': [3.699435,
   4.20655,
   1.31246,
   1.07588625,
   1.31246,
   3.5011219999999996,
   1.003615,
   0,
   0]},
 {'senthomtillis': [-8.291870833333334,
   7.017763333333334,
   3.78693,
   -8.910759166666667,
   3.78693,
   6.744674285714287,
   6.125699999999999,
   19.221848,
   3.882162283333333]},
 {'repkevinyoder': [0.7463941132,
   8.088735,
   0.9529247953333333,
   4.5616075,
   0.9529247953333333,
   2.0793513396,
   1.458124,
   1.861864,
   1.5757359999999998]},
 {'repjohnyarmuth': [-60.222445400000005,
   -64.71197333333333,
   -60.938525,
   -29.089440333333332,
   -60.938525,
 

In [169]:
with open('rep_scores_2016_election.csv', 'w') as f:
    for moc in week9_reps:
        for key, value in moc.items():
            #print(key)
            #print(value)
            row = "%s,%f,%f,%f,%f,%f,%f,%f,%f,%f\n" %(key, value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8])
            f.write(row)